# Model 5: VGG16-SVM-with-Aug

## Transfer Learning: Pre-trained VGG16 + SVM

**Architecture:** VGG16 (remove top) + SVM classifier
- Uses pre-trained ImageNet weights
- Linear SVM kernel
- Input: 224×224×3

**Target:** 98.67% accuracy

In [ ]:
%run 00_utils_and_config.ipynb

In [ ]:
X_train = np.load(CONFIG["processed_data_path"] / "X_train_224.npy").astype("float32")
X_test = np.load(CONFIG["processed_data_path"] / "X_test_224.npy").astype("float32")
y_train = np.load(CONFIG["processed_data_path"] / "y_train.npy")
y_test = np.load(CONFIG["processed_data_path"] / "y_test.npy")

# Convert to PyTorch format (N, C, H, W)
X_train = np.transpose(X_train, (0, 3, 1, 2)) / 255.0
X_test = np.transpose(X_test, (0, 3, 1, 2)) / 255.0

# Normalize using ImageNet stats for VGG16
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
X_train = (torch.from_numpy(X_train).float() - mean) / std
X_test = (torch.from_numpy(X_test).float() - mean) / std

print(f"Data ready: {X_train.shape}, {X_test.shape}")

## Build VGG16 + SVM

In [ ]:
# Load pre-trained VGG16 from torchvision
base_model = torchvision_models.vgg16(pretrained=True)

# Freeze feature extractor
for param in base_model.features.parameters():
    param.requires_grad = False

# Replace classifier with SVM-like layer
class VGG16SVM(nn.Module):
    def __init__(self, base_model, num_classes=2):
        super(VGG16SVM, self).__init__()
        self.features = base_model.features
        self.avgpool = base_model.avgpool
        # VGG16 outputs 512 * 7 * 7 = 25088 features
        self.flatten_size = 25088
        self.fc = nn.Linear(self.flatten_size, num_classes)
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, self.flatten_size)
        x = self.fc(x)
        return x

model = VGG16SVM(base_model, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001, weight_decay=0.01)  # Only train classifier

print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## Train

In [ ]:
# Create datasets and loaders
train_dataset = TensorDataset(X_train, torch.from_numpy(y_train).long())
test_dataset = TensorDataset(X_test, torch.from_numpy(y_test).long())

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

start_time = time.time()
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
best_model_path = str(CONFIG["saved_models_path"] / "model5_vgg16_svm_best.pth")

for epoch in range(10):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / total
    train_acc = correct / total
    
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
    
    print(f"Epoch {epoch+1}/10 - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

training_time = time.time() - start_time

## Evaluate

In [ ]:
best_model = VGG16SVM(base_model, num_classes=2).to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()

start_time = time.time()
all_preds, all_probs = [], []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = best_model(inputs)
        probs = F.softmax(outputs, dim=1)
        all_probs.append(probs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        all_preds.append(predicted.cpu().numpy())

y_pred_proba = np.vstack(all_probs)
y_pred = np.concatenate(all_preds)
testing_time = (time.time() - start_time) * 1000

metrics = calculate_all_metrics(y_test, y_pred, y_pred_proba[:, 1])
print_metrics(metrics, "Model 5: VGG16-SVM")
print(f"Accuracy: {metrics['accuracy']*100:.2f}% (Target: 98.67%)")

In [ ]:
plot_training_history(history, "Model 5: VGG16-SVM", CONFIG["results_path"] / "training_curves" / "model5_training.png")
torch.save(model.state_dict(), CONFIG["saved_models_path"] / "model5_vgg16_svm_final.pth")
results = {"model_name": "VGG16-SVM-with-Aug", "accuracy": float(metrics["accuracy"]), "precision": float(metrics["precision"]), "recall": float(metrics["recall"]), "f1_score": float(metrics["f1_score"]), "training_time_seconds": float(training_time), "testing_time_ms": float(testing_time)}
with open(CONFIG["results_path"] / "model5_results.json", "w") as f: json.dump(results, f, indent=2)
print("✓ VGG16-SVM complete")